In [ ]:
library(nebula)
library(Seurat)

In [ ]:
# Data loading, subset desired cell type and add condition column

object <- readRDS("parentDir/project_postLabel_date.rds")
REP <- subset(object, subset=cellType=="REP")
REP

condition <- plyr::mapvalues(
    x = REP@meta.data$orig.ident, 
    from = c(), 
    to = c()
)
REP@meta.data$condition <- condition

In [ ]:
# Function for properly passing data to and running nebula

RunNebula <- function(seur,form_fixed,sampleCol,cpc=.1)
{
meta=seur@meta.data
dat=seur@assays$RNA@counts
print(dim(dat))
print("Reorder")
dat=dat[,order(meta[,sampleCol])]
meta=meta[order(meta[,sampleCol]),]
print("Run DE!")
df = model.matrix(form_fixed, data=meta)
print(head(df))
print(head(meta[,sampleCol]))

re = nebula(dat,meta[,sampleCol],pred=df,offset=meta$nCount_RNA,cpc=cpc)
return(re)
}

In [ ]:
# Set-up and run nebula

REP@meta.data$condition=factor(REP@meta.data[,"condition"]);REP@meta.data["condition"]=relevel(REP@meta.data[,"condition"],ref="WT")
re <- RunNebula(REP, ~condition, "orig.ident", cpc=0.1)
summ <- re$summary

In [ ]:
# If not adding back rows for filtered genes

summ_filt <- data.frame(gene=summ$gene, logFC=summ$logFC_conditionKO, p_value=summ$p_conditionKO)
summ_filt$padj <- p.adjust(summ_filt$p_value, method="fdr")
summ_filt <- summ_filt[order(summ_filt$p_value),]
head(summ_filt)

In [ ]:
# If adding back rows for filtered genes
allGenes <- rownames(REP)
# Set up results table, add padj and stat column
summ_filt <- data.frame(gene=summ$gene, logFC=summ$logFC_conditionHT, p_value=summ$p_conditionHT)
summ_filt$padj <- p.adjust(summ_filt$p_value, method="fdr")
summ_filt$stat <- -log(summ_filt$p_value)*sign(summ_filt$logFC)

# Find genes filtered for low expression
naGenes <- allGenes[which(!allGenes %in% summ_filt$gene)]

# Make df with NA values for dropped genes
naRows <- data.frame(matrix(ncol=5, nrow=length(naGenes)))
cols = c("gene", "logFC", "p_value", "padj", "stat")
colnames(naRows) <- cols
naRows$gene <- naGenes

# Combine df's, order, check
summ_full <- rbind(summ_filt, naRows)
summ_full <- summ_full[order(summ_full$gene),]
head(summ_full)
dim(summ_full)

In [ ]:
# Write nebula results to csv
write.csv(summ_X, "REP_nebulaDE_DATE.csv", row.names=FALSE)